In [30]:
import pandas as pd
import json
import numpy as np
import pickle

In [2]:
import pandas as pd
import os
import cPickle
from gensim import corpora, models, similarities
#from db_config import engine

In [50]:
df = pd.read_csv('../data/classifier/bosonNLP/bosonNLP-classifier-tuned-huizhou.csv')

In [51]:
import re
regex = re.compile(ur"[^\u4e00-\u9f5aa-zA-Z0-9]") # 中英文和数字

In [5]:
def jieba_cut(df):
    """
    对标题和内容分词
    :return:
    """
    import jieba
    #jieba.load_userdict('Data/userdict.txt')  # 自己准备用户词典，也可不指定  //TODO: ...

    # 1，去掉标点和特殊字符
    # 2，分词
    #df['title_fenci'] = df['title'].apply(lambda x : '|'.join(jieba.cut(regex.sub('',x))))
    df['title_fenci'] = df['title'].apply(lambda x : '|'.join(jieba.cut(x)))
    

In [52]:
jieba_cut(df)

In [53]:
df.head()

,url,domain,title,segment,target,source,title_fenci
0,http://cul.anhuinews.com/system/2009/04/09/002...,anhuinews.com,汪氏后裔品披云徽府菜――越国公宴-徽菜-中安在线-徽文化,汪氏 后裔 品 披 云 徽府菜 ―― 越国 公宴-徽菜 -,1,solr,汪氏|后裔|品披|云徽府|菜|―|―|越国公|宴|-|徽菜|-|中安|在线|-|徽|文化
1,http://cul.anhuinews.com/system/2009/07/01/002...,anhuinews.com,李鸿章大杂烩与八公山豆腐排-徽菜-中安在线-徽文化,李鸿章 大杂烩 与 八 公山 豆腐 排 - 徽菜 -,1,solr,李鸿章|大杂烩|与|八|公山|豆腐|排|-|徽菜|-|中安|在线|-|徽|文化
2,http://cul.anhuinews.com/system/2009/07/14/002...,anhuinews.com,"徽派版画雕刻-徽派版画,徽学流派-中安在线-徽文化","徽派 版画 雕刻-徽派 版画 , 徽学 流派 -",1,solr,"徽派|版画|雕刻|-|徽派|版画|,|徽学|流派|-|中安|在线|-|徽|文化"
3,http://cul.anhuinews.com/system/2009/10/20/002...,anhuinews.com,砚雕造神奇 访徽州民间工艺大师王祖伟-大师-中安在线-徽文化,砚雕 造 神奇 访 徽州 民间 工艺 大师 王祖伟 - 大师 -,1,solr,砚|雕造|神奇| |访|徽州|民间工艺|大师|王祖伟|-|大师|-|中安|在线|-|徽|文化
4,http://cul.anhuinews.com/system/2014/04/17/006...,anhuinews.com,徽州区民俗文物博物馆建成免费开放 徽文化频道 中安在线,徽州区 民俗 文物 博物馆 建成 免费 开放,1,solr,徽州区|民俗|文物|博物馆|建成|免费|开放| |徽|文化频道| |中安|在线


In [54]:
def remove_low_freq_word(texts, times=1):
    """
    去掉低频词
    :param times:出现次数
    :return:
    """
    all_tokens = sum(texts, [])
    title_token_once = set(word for word in set(all_tokens) if all_tokens.count(word) == times)

    texts_result = [[word for word in text if word not in title_token_once] for text in texts]
    return texts_result


In [55]:
texts = []
for ix, row in df.iterrows():
    texts_cuts = row['title_fenci'].split('|')
    texts.append(texts_cuts)

texts = remove_low_freq_word(texts)

In [56]:
with open('../data/bag_of_words/huizou_classified_titles.txt', 'wb') as fp:
    pickle.dump(texts, fp)

In [58]:
with open ('../data/bag_of_words/huizou_classified_titles.txt', 'rb') as fp:
    texts_load = pickle.load(fp)

In [59]:
#bag of words and map token to id
dictionary = corpora.Dictionary(texts_load)
corpus = [dictionary.doc2bow(text) for text in texts_load]

In [60]:
#tfidf 
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]


In [61]:
# Lsi最核心的意义是将训练文档向量组成的矩阵SVD分解，并做了一个秩为2的近似SVD分解
lsi = models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=10)

In [62]:
# 有了LSI模型，建立索引
index = similarities.MatrixSimilarity(lsi[corpus])

In [64]:
# 计算某一篇文档的相似度
tl_bow = dictionary.doc2bow(['' ])
#tl_bow = dictionary.doc2bow(df.ix[5001, 'title_fenci'].split('|'))
tl_lsi = lsi[tl_bow]
# print tl_lsi
sims = index[tl_lsi]
# print sims

sort_sims = sorted(enumerate(sims), key=lambda item: -item[1]) #

for t in sort_sims[:10]:
    print df.ix[t[0], 'title'], t[1]
        


歙县太白楼 1.0
歙县北岸 0.999969
歙县小溪 0.999953
歙县鲁人书画院画展 0.999937
歙县南谯楼 0.999581
航拍歙县宣传片 0.99958
歙县千载渔梁坝 0.999333
歙县深渡镇漳潭 0.99925
歙县三阳乡叶村叠罗汉 0.998978
歙县深渡码头 0.99897


In [19]:
print df.ix[5001, 'title_fenci'].split('|')[0]

割


In [67]:
print len(texts), df.shape

7020 (7020, 7)


In [38]:
print len(dictionary), type(dictionary)

16224 <class 'gensim.corpora.dictionary.Dictionary'>


In [65]:
print dictionary

Dictionary(4350 unique tokens: [u'', u'\u6302', u'\u575a\u5b88', u'\u7a7f\u8d8a\u65f6\u7a7a', u'\u5982\u540c']...)
